In [2]:
# %% General IQ resampler with widgets (auto format + header Fs + optional recenter + analyze)
import os
from fractions import Fraction

import numpy as np
from scipy.signal import resample_poly
from scipy.io import wavfile
import ipywidgets as W
from IPython.display import display

# ---------- 対象ファイル列挙 ----------
def list_data_files(extensions=(".bin", ".dat", ".iq", ".wav")):
    files = [
        f for f in os.listdir(".")
        if os.path.isfile(f) and f.lower().endswith(extensions)
    ]
    files.sort()
    return files

# ---------- I/Q 読み込み ----------
def read_iq(path, fmt):
    """
    戻り値:
      x         : complex64 の 1次元 I/Q 列
      header_fs : ヘッダ等から取得したサンプリング周波数[Hz] (なければ None)
      aux       : フォーマット固有情報 (WAV の dtype など)
    """
    if fmt == "int16_iq":
        raw = np.fromfile(path, dtype=np.int16)
        if raw.size % 2:
            raw = raw[:-1]
        I = raw[0::2].astype(np.float32)
        Q = raw[1::2].astype(np.float32)
        x = I + 1j * Q
        return x.astype(np.complex64), None, None

    elif fmt == "complex64":
        x = np.fromfile(path, dtype=np.complex64)
        return x.astype(np.complex64), None, None

    elif fmt == "wav_stereo":
        fs, data = wavfile.read(path)   # fs: int (Hz), data: (N, C)
        if data.ndim != 2 or data.shape[1] < 2:
            raise ValueError("WAV は少なくとも 2ch (L=I, R=Q) が必要です。")
        I = data[:, 0].astype(np.float32)
        Q = data[:, 1].astype(np.float32)
        x = I + 1j * Q
        return x.astype(np.complex64), float(fs), data.dtype

    else:
        raise ValueError(f"未知のフォーマット: {fmt}")

# ---------- フォーマット自動判定 ----------
def auto_format_for(path):
    ext = os.path.splitext(path)[1].lower()
    if ext == ".wav":
        return "wav_stereo"
    else:
        return "int16_iq"

# ---------- I/Q 書き込み ----------
def write_iq(path, x, fmt, header_fs=None, aux=None, fs_out=None):
    if fmt == "int16_iq":
        xr = np.real(x)
        xi = np.imag(x)

        mx = max(
            float(np.max(np.abs(xr))) if xr.size else 0.0,
            float(np.max(np.abs(xi))) if xi.size else 0.0,
            1e-12,
        )
        scale = 32767.0 / mx if mx > 32767.0 else 1.0

        I16 = np.clip(np.round(xr * scale), -32768, 32767).astype(np.int16)
        Q16 = np.clip(np.round(xi * scale), -32768, 32767).astype(np.int16)

        inter = np.empty(I16.size + Q16.size, dtype=np.int16)
        inter[0::2] = I16
        inter[1::2] = Q16
        inter.tofile(path)

    elif fmt == "complex64":
        np.asarray(x, dtype=np.complex64).tofile(path)

    elif fmt == "wav_stereo":
        if fs_out is None:
            raise ValueError("wav_stereo 出力には fs_out (Hz) が必要です。")
        if aux is None:
            raise ValueError("wav_stereo 出力には元の dtype(aux) が必要です。")

        yI = np.real(x)
        yQ = np.imag(x)

        if np.issubdtype(aux, np.integer):
            info = np.iinfo(aux)
            max_val = info.max
            min_val = info.min
            mx = max(
                float(np.max(np.abs(yI))) if yI.size else 0.0,
                float(np.max(np.abs(yQ))) if yQ.size else 0.0,
                1e-12,
            )
            scale = max_val / mx if mx > max_val else 1.0

            Iq = np.stack([
                np.clip(np.round(yI * scale), min_val, max_val).astype(aux),
                np.clip(np.round(yQ * scale), min_val, max_val).astype(aux),
            ], axis=1)
        else:
            Iq = np.stack([yI.astype(aux), yQ.astype(aux)], axis=1)

        wavfile.write(path, int(round(fs_out)), Iq)

    else:
        raise ValueError(f"未知のフォーマット: {fmt}")

# ---------- サフィックス ----------
def suffix_from_fs(mhz):
    s = f"{mhz:.6f}".rstrip("0").rstrip(".")
    return s.replace(".", "p") + "MHz"

# ---------- ウィジェット ----------
files = list_data_files()
file_dd = W.Dropdown(options=files, description="File:")
reload_btn = W.Button(description="🔄", tooltip="Refresh file list")

analyze_btn = W.Button(description="Analyze", tooltip="Detect format & sampling rate")

fmt_dd = W.Dropdown(
    options=[
        ("Select format (auto by extension)", "auto"),
        ("int16 I/Q interleaved (I,Q,I,Q,...)", "int16_iq"),
        ("complex64 continuous (complex64)", "complex64"),
        ("WAV stereo (L=I, R=Q)", "wav_stereo"),
    ],
    description="Format:",
)

fs_from = W.FloatText(description="Input sampling rate [MHz]", value=0.0)
fs_to   = W.FloatText(description="Output sampling rate [MHz]", value=1.0)

fc_in   = W.FloatText(description="Input center [MHz]", value=0.0)
fc_out  = W.FloatText(description="Recenter to [MHz]", value=0.0)

info_label = W.HTML(value="")

run_btn = W.Button(description="Run", button_style="primary")
log     = W.Output()

info_hint = W.HTML(
    value=(
        "<span style='color:gray'>"
        "Format=auto のときは拡張子から判定。WAV は header の sampling rate を優先して使います。"
        "Analyze を押すと format / total samples / duration を表示します。"
        "</span>"
    )
)

# ---------- コールバック ----------
def on_reload(_):
    file_dd.options = list_data_files()
reload_btn.on_click(on_reload)

def on_analyze(_):
    log.clear_output()
    with log:
        path = file_dd.value
        if not path:
            print("Error: no input file.")
            return

        # auto のときはここで決めてしまう
        if fmt_dd.value == "auto":
            fmt = auto_format_for(path)
            fmt_dd.value = fmt
        else:
            fmt = fmt_dd.value

        print(f"Analyze: {path}")
        print(f"Detected / selected format: {fmt}")

        try:
            x, header_fs, aux = read_iq(path, fmt)
        except Exception as e:
            print(f"Read error: {e}")
            return

        N = x.size
        fs_hz = None

        if header_fs is not None:
            fs_hz = header_fs
            fs_from.value = header_fs / 1e6
            print(f"Sampling rate from header: {fs_from.value:.6f} MHz")
        elif fs_from.value > 0:
            fs_hz = fs_from.value * 1e6
            print(f"Sampling rate from widget: {fs_from.value:.6f} MHz")
        else:
            print("Sampling rate is unknown (no header, widget=0).")

        if fs_hz is not None:
            dur = N / fs_hz
            txt = (
                f"Format={fmt}, Total samples={N}, "
                f"Duration={dur:.6f} s (sampling rate={fs_hz/1e6:.6f} MHz)"
            )
        else:
            txt = f"Format={fmt}, Total samples={N}, Duration=unknown (sampling rate not set)"

        info_label.value = f"<span style='color:gray'>{txt}</span>"

analyze_btn.on_click(on_analyze)

def do_resample(_):
    log.clear_output()
    with log:
        if not file_dd.value:
            print("Error: no input file.")
            return
        if fs_to.value <= 0:
            print("Error: Output sampling rate [MHz] must be > 0.")
            return

        in_path = file_dd.value

        if fmt_dd.value == "auto":
            fmt = auto_format_for(in_path)
        else:
            fmt = fmt_dd.value

        print(f"Input file : {in_path}")
        print(f"Format     : {fmt}")

        try:
            x, header_fs, aux = read_iq(in_path, fmt)
        except Exception as e:
            print(f"Read error: {e}")
            return

        fs_in = None
        fs_in_user  = fs_from.value * 1e6
        fs_out      = fs_to.value   * 1e6

        if header_fs is not None:
            fs_in = header_fs
            header_mhz = header_fs / 1e6
            print(f"Header sampling rate : {header_mhz:.6f} MHz (used as input)")
            if fs_from.value > 0 and abs(header_fs - fs_in_user) > 1.0:
                print(
                    f"Warning: header={header_mhz:.6f} MHz vs widget={fs_from.value:.6f} MHz. "
                    "Header value is used."
                )
        else:
            if fs_from.value <= 0:
                print("Error: Input sampling rate [MHz] must be > 0 when no header is available.")
                return
            fs_in = fs_in_user
            print(f"Input sampling rate : {fs_from.value:.6f} MHz (from widget)")

        print(f"Output sampling rate: {fs_to.value:.6f} MHz")

        if not np.isfinite(x).all():
            print("Warning: input data contains NaN/Inf. Aborting.")
            return

        # recenter
        fc_in_mhz  = float(fc_in.value) if fc_in.value is not None else 0.0
        fc_out_mhz = float(fc_out.value) if fc_out.value is not None else 0.0

        if abs(fc_in_mhz - fc_out_mhz) > 1e-9:
            if fs_in is None:
                print("Warning: Fs is unknown; cannot perform recentering.")
            else:
                f_in  = fc_in_mhz  * 1e6
                f_out = fc_out_mhz * 1e6
                f_shift = f_in - f_out
                print(
                    f"Recenter: {fc_in_mhz:.6f} MHz -> {fc_out_mhz:.6f} MHz "
                    f"(Δ = {f_shift/1e6:.6f} MHz)"
                )
                n = np.arange(x.size, dtype=np.float64)
                x = x * np.exp(-1j * 2 * np.pi * f_shift / fs_in * n)
        else:
            if fc_in_mhz != 0.0:
                print(f"Center freq: {fc_in_mhz:.6f} MHz (no recentering).")

        frac = Fraction(float(fs_out) / float(fs_in)).limit_denominator(1000)
        up, down = frac.numerator, frac.denominator
        print(f"Ratio     : {fs_out/fs_in:.9f} ≈ {up}/{down}")
        print("Running resample_poly...")

        try:
            y = resample_poly(x, up, down, window=("kaiser", 8.6))
        except Exception as e:
            print(f"resample_poly error: {e}")
            return

        stem, ext = os.path.splitext(in_path)
        out_path = f"{stem}_Fs{suffix_from_fs(fs_to.value)}{ext}"

        try:
            write_iq(out_path, y, fmt, header_fs=header_fs, aux=aux, fs_out=fs_out)
        except Exception as e:
            print(f"Write error: {e}")
            return

        print(f"Saved     : {out_path}")
        print("Done.")

run_btn.on_click(do_resample)

# ---------- レイアウト表示 ----------
display(
    W.HBox([file_dd, reload_btn]),
    analyze_btn,
    fmt_dd,
    W.HBox([fs_from, fs_to]),
    W.HBox([fc_in, fc_out]),
    info_label,
    info_hint,
    run_btn,
    log,
)


Button(description='Analyze', style=ButtonStyle(), tooltip='Detect format & sampling rate')

Dropdown(description='Format:', options=(('Select format (auto by extension)', 'auto'), ('int16 I/Q interleave…

HTML(value='')

HTML(value="<span style='color:gray'>Format=auto のときは拡張子から判定。WAV は header の sampling rate を優先して使います。Analyze を押…

Button(button_style='primary', description='Run', style=ButtonStyle())

Output()

In [5]:
# %% 汎用 I/Q スペクトラムビューア（フォーマット自動判定版・Warning抑止版）
import os
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as W
from IPython.display import display

from scipy.io import wavfile
from scipy.io.wavfile import WavFileWarning
import warnings

# ← ここで一括で WavFileWarning を抑止
warnings.filterwarnings("ignore", category=WavFileWarning)

# ---------- ファイルリスト ----------
def list_data_files(extensions=(".bin", ".dat", ".iq", ".npy", ".wav")):
    files = [
        f for f in os.listdir(".")
        if os.path.isfile(f) and f.lower().endswith(extensions)
    ]
    files.sort()
    return files

# ---------- I/Q 読み込み（フォーマット自動判定） ----------
def read_iq_auto(path):
    ext = os.path.splitext(path)[1].lower()

    # ---- WAV ----
    if ext == ".wav":
        fs, data = wavfile.read(path)    # WavFileWarning は上で抑止済み
        if data.ndim != 2 or data.shape[1] < 2:
            raise ValueError("WAV は 2ch (L=I, R=Q) が必要です。")
        I = data[:, 0].astype(np.float32)
        Q = data[:, 1].astype(np.float32)
        x = I + 1j * Q
        return x.astype(np.complex64), float(fs), "wav_stereo"

    # ---- NPY (complex) ----
    if ext == ".npy":
        arr = np.load(path)
        if not np.iscomplexobj(arr):
            raise ValueError(".npy が複素配列ではありません。I/Qとして扱えません。")
        return arr.astype(np.complex64), None, "complex_npy"

    # ---- その他 → int16 I/Q 交互とみなす ----
    raw = np.fromfile(path, dtype=np.int16)
    if raw.size % 2:
        raw = raw[:-1]
    I = raw[0::2].astype(np.float32)
    Q = raw[1::2].astype(np.float32)
    x = I + 1j * Q
    return x.astype(np.complex64), None, "int16_iq"

# ---------- ウィジェット ----------
files = list_data_files()
file_dd = W.Dropdown(options=files, description="File:", layout=W.Layout(width="70%"))
reload_btn = W.Button(description="🔄", tooltip="ファイル一覧更新")

fs_box = W.FloatText(description="Sampling rate Fs [MHz]", value=0.0)
fc_box = W.FloatText(description="Center freq [MHz]", value=0.0)

nfft_box = W.IntText(description="NFFT", value=65536)
win_dd = W.Dropdown(
    options=[("Hann", "hann"), ("Rect", "rect")],
    description="Window:",
    value="hann",
)

run_btn = W.Button(description="Draw spectrum", button_style="primary")
out = W.Output()

info = W.HTML(
    value="<span style='color:gray'>Format は拡張子から自動判定します。"
          "Sampling rate Fs / Center freq は基本的に手入力（WAV はヘッダ Fs も参照）。</span>"
)

def on_reload(_):
    file_dd.options = list_data_files()

reload_btn.on_click(on_reload)

def on_run(_):
    out.clear_output()
    with out:
        if not file_dd.value:
            print("エラー: 対象ファイルがありません。")
            return

        path = file_dd.value

        try:
            x, header_fs, fmt_name = read_iq_auto(path)
        except Exception as e:
            print(f"読み込みエラー: {e}")
            return

        print(f"File: {path}")
        print(f"Detected format: {fmt_name}")

        if x.size == 0:
            print("エラー: データ長が 0 です。")
            return

        NFFT = max(1, int(nfft_box.value))
        N = min(NFFT, x.size)

        # Fs 決定
        fs_user_mhz = float(fs_box.value) if fs_box.value is not None else 0.0
        fs_user = fs_user_mhz * 1e6 if fs_user_mhz > 0 else None
        fs = None

        if fs_user is not None:
            fs = fs_user
            if header_fs is not None and abs(header_fs - fs) > 1.0:
                print(
                    f"警告: WAV ヘッダ Fs={header_fs/1e6:.6f} MHz と "
                    f"入力 Fs={fs_user_mhz:.6f} MHz が一致しません。"
                )
        else:
            if header_fs is not None:
                fs = header_fs
                print(f"Info: Using WAV header Fs={fs/1e6:.6f} MHz.")
            else:
                print("Warning: Fs unknown. X-axis will be normalized frequency.")

        fc_mhz = float(fc_box.value) if fc_box.value is not None else 0.0
        fc = fc_mhz * 1e6

        if win_dd.value == "hann":
            w = np.hanning(N).astype(np.float32)
        else:
            w = np.ones(N, np.float32)

        spec = np.fft.fftshift(np.fft.fft(x[:N] * w, n=N)) / (np.sum(w) / N)
        mag = 20*np.log10(np.maximum(np.abs(spec), 1e-20))
        mag -= mag.max()

        if fs is not None:
            frel = np.fft.fftshift(np.fft.fftfreq(N, d=1/fs))  # Hz, -Fs/2～+Fs/2
            faxis = (fc + frel) / 1e6                           # MHz
            xlabel = "Frequency [MHz]"
            title_fs = f"Sampling rate Fs={fs/1e6:.3f} MHz"
        else:
            frel = np.fft.fftshift(np.fft.fftfreq(N, d=1.0))
            faxis = frel
            xlabel = "Normalized frequency [cycles/sample] (Fs unknown)"
            title_fs = "Sampling rate Fs=Unknown"

        plt.figure(figsize=(10, 5))
        plt.plot(faxis, mag)
        plt.grid(True, alpha=0.3)
        title = (
            f"Spectrum: {path}\n"
            f"Format={fmt_name}, {title_fs}, Center freq={fc/1e6:.3f} MHz, "
            f"N={N}, Window={win_dd.label}"
        )
        plt.title(title)
        plt.xlabel(xlabel)
        plt.ylabel("Relative Magnitude [dB]")
        plt.show()

run_btn.on_click(on_run)

display(
    W.HBox([file_dd, reload_btn]),
    W.HBox([fs_box, fc_box]),
    W.HBox([nfft_box, win_dd]),
    info,
    run_btn,
    out,
)


HTML(value="<span style='color:gray'>Format は拡張子から自動判定します。Sampling rate Fs / Center freq は基本的に手入力（WAV はヘッダ Fs …

Button(button_style='primary', description='Draw spectrum', style=ButtonStyle())

Output()

In [7]:
# %% IQ Trimmer with widgets (sample/time selectable)
# - Supports:
#    * int16 I/Q interleaved (I,Q,I,Q,...)  [default for .bin/.dat/.iq]
#    * complex64 continuous                 [manual select]
#    * complex NPY (.npy, complex)          [manual or auto]
#    * WAV stereo (L=I, R=Q)                [.wav]
# - 「Analyze」を押すと:
#    * フォーマット自動判定 (auto の場合)
#    * 全サンプル数表示
#    * サンプリングレートが分かれば全体時間も表示
# - 「Trim by」で「samples / time」を選択
# - 「Cut & save」で実際にトリム＆保存

import os
import numpy as np
from scipy.io import wavfile
import ipywidgets as W
from IPython.display import display

# ----------------- helpers -----------------

def list_data_files(extensions=(".bin", ".dat", ".iq", ".wav", ".npy")):
    files = [
        f for f in os.listdir(".")
        if os.path.isfile(f) and f.lower().endswith(extensions)
    ]
    files.sort()
    return files

def auto_format_for(path):
    ext = os.path.splitext(path)[1].lower()
    if ext == ".wav":
        return "wav_stereo"
    elif ext == ".npy":
        return "complex_npy"
    else:
        return "int16_iq"   # default for raw IQ

def read_iq_for_trim(path, fmt):
    """
    読み込み専用。
    returns:
      x         : complex64 1D array (I + jQ)
      header_fs : sample rate [Hz] if known, else None
      aux       : extra info (e.g. WAV dtype)
    """
    ext = os.path.splitext(path)[1].lower()

    if fmt == "wav_stereo":
        fs, data = wavfile.read(path)  # fs: Hz, data: (N, C)
        if data.ndim != 2 or data.shape[1] < 2:
            raise ValueError("WAV must have at least 2 channels (L=I, R=Q).")
        I = data[:, 0].astype(np.float32)
        Q = data[:, 1].astype(np.float32)
        x = I + 1j * Q
        return x.astype(np.complex64), float(fs), data.dtype

    if fmt == "complex_npy" and ext == ".npy":
        arr = np.load(path)
        if not np.iscomplexobj(arr):
            raise ValueError(".npy array is not complex.")
        return arr.astype(np.complex64), None, None

    if fmt == "complex64":
        arr = np.fromfile(path, dtype=np.complex64)
        return arr.astype(np.complex64), None, None

    # default: int16 IQ interleaved
    raw = np.fromfile(path, dtype=np.int16)
    if raw.size % 2:
        raw = raw[:-1]
    I = raw[0::2].astype(np.float32)
    Q = raw[1::2].astype(np.float32)
    x = I + 1j * Q
    return x.astype(np.complex64), None, None

def write_iq_trim(path, x, fmt, header_fs=None, aux=None):
    """
    x is complex64 trimmed segment.
    Writes in the same format as original, without rescaling.
    """
    ext = os.path.splitext(path)[1].lower()

    if fmt == "wav_stereo":
        if header_fs is None:
            raise ValueError("Need header_fs for WAV output.")
        if aux is None:
            raise ValueError("Need dtype(aux) for WAV output.")

        I = np.real(x)
        Q = np.imag(x)

        if np.issubdtype(aux, np.integer):
            Iq = np.stack([I.astype(aux), Q.astype(aux)], axis=1)
        else:
            Iq = np.stack([I.astype(aux), Q.astype(aux)], axis=1)

        wavfile.write(path, int(round(header_fs)), Iq)
        return

    if fmt in ("complex64", "complex_npy") and ext == ".npy":
        # keep NPY format
        np.save(path, np.asarray(x, dtype=np.complex64))
        return

    if fmt in ("complex64", "complex_npy") and ext != ".npy":
        np.asarray(x, dtype=np.complex64).tofile(path)
        return

    # int16 IQ interleaved
    I16 = np.real(x).astype(np.int16)
    Q16 = np.imag(x).astype(np.int16)
    inter = np.empty(I16.size + Q16.size, dtype=np.int16)
    inter[0::2] = I16
    inter[1::2] = Q16
    inter.tofile(path)

# ----------------- widgets -----------------

files = list_data_files()
file_dd = W.Dropdown(options=files, description="File:", layout=W.Layout(width="70%"))
reload_btn = W.Button(description="🔄", tooltip="Refresh file list")

fmt_dd = W.Dropdown(
    options=[
        ("Select format (auto by extension)", "auto"),
        ("int16 I/Q interleaved (I,Q,I,Q,...)", "int16_iq"),
        ("complex64 continuous (raw file)", "complex64"),
        ("complex NPY (numpy .npy)", "complex_npy"),
        ("WAV stereo (L=I, R=Q)", "wav_stereo"),
    ],
    description="Format:",
    layout=W.Layout(width="70%"),
)

fs_box = W.FloatText(description="Sampling rate [MHz]", value=0.0)
analyze_btn = W.Button(description="Analyze", button_style="info")
info_label = W.HTML(value="")

# どちらでトリムするか選ぶ
mode_radio = W.RadioButtons(
    options=[
        ("Use sample indices", "samples"),
        ("Use time [s]", "time"),
    ],
    description="Trim by:",
)

# sample 指定
start_box = W.IntText(description="Start sample", value=0)
end_box   = W.IntText(description="End sample", value=0)

# time 指定
start_time_box = W.FloatText(description="Start time [s]", value=0.0)
end_time_box   = W.FloatText(description="End time [s]",   value=0.0)

run_btn = W.Button(description="Cut & save", button_style="primary")
log = W.Output()

# データキャッシュ
_current = {
    "path": None,
    "fmt": None,
    "x": None,
    "header_fs": None,
    "aux": None,
}

# ----------------- callbacks -----------------

def load_and_analyze(path):
    """選択されたファイルを読み込んで _current を更新し、情報を表示"""
    fmt_sel = fmt_dd.value
    if fmt_sel == "auto":
        fmt = auto_format_for(path)
    else:
        fmt = fmt_sel

    x, header_fs, aux = read_iq_for_trim(path, fmt)

    _current["path"] = path
    _current["fmt"] = fmt
    _current["x"] = x
    _current["header_fs"] = header_fs
    _current["aux"] = aux

    N = x.size
    start_box.value = 0
    end_box.value = N

    # sampling rate の決定
    fs_hz = None
    if header_fs is not None:
        fs_hz = header_fs
        # WAV の場合はヘッダ値を fs_box に反映してやってもいい
        fs_box.value = header_fs / 1e6
    elif fs_box.value > 0:
        fs_hz = fs_box.value * 1e6

    if fs_hz is not None:
        dur = N / fs_hz
        info = (
            f"Format: {fmt}, Total samples: {N}, "
            f"Duration: {dur:.6f} s (Sampling rate={fs_hz/1e6:.6f} MHz)"
        )
    else:
        info = f"Format: {fmt}, Total samples: {N}, Duration: unknown (sampling rate not set)"

    info_label.value = f"<span style='color:gray'>{info}</span>"

def on_reload(_):
    file_dd.options = list_data_files()

reload_btn.on_click(on_reload)

def on_file_change(change):
    # ファイル変更時はまだ自動で読み込まず、Analyze で読む運用にするならここは何もしなくてもよい
    pass

file_dd.observe(on_file_change, names="value")

def on_analyze(_):
    log.clear_output()
    with log:
        path = file_dd.value
        if not path:
            print("Error: no input file selected.")
            return
        print(f"Analyzing: {path}")
        try:
            load_and_analyze(path)
        except Exception as e:
            print(f"Read error: {e}")

analyze_btn.on_click(on_analyze)

def on_run(_):
    log.clear_output()
    with log:
        path = file_dd.value
        if not path:
            print("Error: no input file selected.")
            return

        if _current["path"] != path or _current["x"] is None:
            print("Info: file changed or not analyzed yet. Running Analyze first...")
            try:
                load_and_analyze(path)
            except Exception as e:
                print(f"Read error: {e}")
                return

        x = _current["x"]
        fmt = _current["fmt"]
        header_fs = _current["header_fs"]
        aux = _current["aux"]

        N = x.size

        # サンプリングレート決定（time モードでは必須）
        fs_hz = None
        if header_fs is not None:
            fs_hz = header_fs
        elif fs_box.value > 0:
            fs_hz = fs_box.value * 1e6

        mode = mode_radio.value

        if mode == "samples":
            s = int(start_box.value)
            e = int(end_box.value)
            if s < 0 or e <= s or e > N:
                print(f"Error: invalid sample range. 0 <= start < end <= {N}")
                return
            seg = x[s:e]
            length = seg.size
            print(f"Using sample range: {s} -> {e} (length={length})")
            # 時間表示（分かる場合だけ）
            if fs_hz is not None:
                t_start = s / fs_hz
                t_end = e / fs_hz
                print(f"(This corresponds to {t_start:.6f} s -> {t_end:.6f} s)")
        else:  # mode == "time"
            if fs_hz is None:
                print("Error: sampling rate is unknown. Cannot use time mode.")
                return
            t_start = float(start_time_box.value)
            t_end   = float(end_time_box.value)
            if t_start < 0 or t_end <= t_start:
                print("Error: invalid time range. Need 0 <= start < end.")
                return
            s = int(round(t_start * fs_hz))
            e = int(round(t_end   * fs_hz))
            if s < 0:
                s = 0
            if e > N:
                e = N
            if e <= s:
                print("Error: time range too short after conversion to samples.")
                return
            seg = x[s:e]
            length = seg.size
            print(
                f"Using time range: {t_start:.6f} s -> {t_end:.6f} s "
                f"(samples {s} -> {e}, length={length})"
            )

        stem, ext = os.path.splitext(path)
        out_path = f"{stem}_Trim{str(s).zfill(8)}_{length}{ext}"

        print(f"Input file : {path}")
        print(f"Format     : {fmt}")
        print(f"Total N    : {N}")
        print(f"Output file: {out_path}")

        # 書き込み
        try:
            write_iq_trim(out_path, seg, fmt, header_fs=header_fs, aux=aux)
        except Exception as e:
            print(f"Write error: {e}")
            return

        if fs_hz is not None:
            t0 = s / fs_hz
            t1 = e / fs_hz
            print(f"Final time range: {t0:.6f} s -> {t1:.6f} s")
        else:
            print("Final time range: unknown (sampling rate not set)")

        print("Done.")

run_btn.on_click(on_run)

# ----------------- show UI -----------------

display(
    W.HBox([file_dd, reload_btn]),
    fmt_dd,
    fs_box,
    analyze_btn,
    info_label,
    mode_radio,
    W.HBox([start_box, end_box]),
    W.HBox([start_time_box, end_time_box]),
    run_btn,
    log,
)


Dropdown(description='Format:', layout=Layout(width='70%'), options=(('Select format (auto by extension)', 'au…

FloatText(value=0.0, description='Sampling rate [MHz]')

Button(button_style='info', description='Analyze', style=ButtonStyle())

HTML(value='')

RadioButtons(description='Trim by:', options=(('Use sample indices', 'samples'), ('Use time [s]', 'time')), va…

Button(button_style='primary', description='Cut & save', style=ButtonStyle())

Output()